In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import os
import yaml
import numpy as np
from copy import deepcopy

# What and why?

Per default the pycomplexes convert tool assumes the pH in your simulation to be pH = 7.0.
We provided a tool (**ph**) to change the charges of your coarse-grained in accordance to pH values different from 7.0.
We assume the amino acids to be quasi free and apply the *Henderson-Hasselbalch* equation. The **ph** tool applies changes to
an existing cplx file.

# Change sidechain charges in cplx structure file

We prepared a very artificial structure file for this tutorial. 
We can make use of the command line to quickly change the charges in the file in accordance to a certain pH. Let's prepare structures at a pH = 4

In [ ]:
!pycomplexes ph test.cplx 4 -o pH4.cplx

Optionally to the CLI tool you can also directly make use of the pycomplexes python library. As an example we create structures for a pH 4 to 10

In [ ]:
from pycomplexes import ph

We load the prepared structure file:

In [ ]:
with open("test.cplx") as f:
    cplx_struct = yaml.load(f)

We create structures for different pHs

In [ ]:
pHs = np.arange(4, 10, 0.5)
modified_cplx_structs = [deepcopy(ph.change_charges(cplx_struct, ph=pH)) for pH in pHs]

and write the structures into separate cplx files

In [ ]:
for pH, modified_struc in zip(pHs, modified_cplx_structs):
    fname = "pH{}.cplx".format(pH)
    with open(fname, "w") as f:
        yaml.dump(modified_struc, f)

We wrote a couple of files now. Check yourself.

Let's see how the charges in the structure files have changed
Here we specifically look at the changed charges for a single HIS in the cplx

In [ ]:
top_idx = 1
domain_idx = 3
charges = []
for pH, modified_struc in zip(pHs, modified_cplx_structs):
    resname = modified_struc["topologies"][top_idx]["domains"][domain_idx]["beads"][0]
    charge = modified_struc["topologies"][top_idx]["domains"][domain_idx]["charges"][0]
    charges.append(charge)

In [ ]:
f,axs = plt.subplots(figsize=plt.figaspect(.5))
axs.axhline( .5, c="r", ls="--", alpha=.5)
axs.axvline(6.5, c="k", ls="--", alpha=.5, label="pKA=6.5")
axs.plot(pHs, charges, "o", label="histidine")
axs.set(xlabel="pH",
        ylabel="magn. of sidechain charge")
axs.legend();

Try other residues in the example file and see how the charges of other residue types changes with the pH.

# Use custom pKA values for the residues

In [ ]:
# lets make a copy of the default pKA values
charged_sidechains = deepcopy(ph.CHARGED_SIDECHAINS)

In [ ]:
# default pKA values and information, whether the sidechain is acidic or basic (charge is -1 or +1)
for resname, subdict in charged_sidechains.items():
    print("residue name: {}".format(resname))
    for key, val in subdict.items():
        print("\t{}:\t {:>4.1f}".format(key, val))

We want to apply a pKA-value of 6.0 to residues of type histidine, which is different than the default value. 
Therefore, we change the dictionary with the pKA information in the following way. 
We leave all other pKAs untouched. 

In [ ]:
charged_sidechains["HIS"]["pK"] = 6.0

We redo the procedure above with the modified values.

In [ ]:
modified_cplx_structs = [deepcopy(ph.change_charges(cplx_struct, ph=pH, charged_sidechains=charged_sidechains)) for pH in pHs]

top_idx = 1
domain_idx = 3
charges_new = []
for pH, modified_struc in zip(pHs, modified_cplx_structs):
    resname = modified_struc["topologies"][top_idx]["domains"][domain_idx]["beads"][0]
    charge = modified_struc["topologies"][top_idx]["domains"][domain_idx]["charges"][0]
    charges_new.append(charge)

We visualize the change of the titration curve:

In [ ]:
f,axs = plt.subplots(figsize=plt.figaspect(0.5))
axs.axhline( .5, c="r", ls="--", alpha=.5)
axs.axvline(6.5, c="k", ls="--", alpha=.2, label="pKA=6.5")
axs.axvline(6.0, c="k", ls="--", alpha=.5, label="pKA=6.0")
axs.plot(pHs, charges, "o", label="histidine: default", alpha=.3)
axs.plot(pHs, charges_new, "o", label="histidine: modified")
axs.set(xlabel="pH",
        ylabel="magn. of sidechain charge")
axs.legend(ncol=2);

We can also save the dictionary >charged_sidechains< in a file and use custom pKA values with the command-line tool with the flag: -pk